## Packages download :

In [ ]:
!pip install open3d scipy spektral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 408.6 MB 30 kB/s 
     |████████████████████████████████| 140 kB 91.5 MB/s 
     |████████████████████████████████| 3.1 MB 66.9 MB/s 
     |████████████████████████████████| 8.8 MB 34.3 MB/s 
     |████████████████████████████████| 1.1 MB 73.3 MB/s 
     |████████████████████████████████| 54 kB 3.6 MB/s 
     |████████████████████████████████| 344 kB 77.1 MB/s 
     |████████████████████████████████| 9.7 MB 80.7 MB/s 
     |████████████████████████████████| 423 kB 75.2 MB/s 
     |████████████████████████████████| 1.6 MB 59.3 MB/s 
     |████████████████████████████████| 1.2 MB 67.4 MB/s 
     |████████████████████████████████| 59 kB 7.3 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 100.4 MB/s 
     |████████████████████████████████| 80 kB 11.2 MB/s 
     |███████████████████████████████

## Libraries and Loading the point clouds :

In [ ]:
import open3d as o3d
import random
import tensorflow as tf
import numpy as np
import spektral
from tensorflow.keras.layers import Dropout, Input
from tensorflow import keras
from tensorflow.keras import layers
from scipy import sparse
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool
from spektral.data import BatchLoader
from spektral.data.loaders import Loader
from spektral.layers import ChebConv
from spektral.data import Dataset
from scipy.spatial import distance_matrix
from spektral.data import Graph
import os
import glob
from scipy.sparse import save_npz
from spektral.layers import GATConv

path ="/content/data/clean_train/predlift/boxer/original.ply"

In [ ]:
# pcd = o3d.io.read_point_cloud("/content/data/clean_train/predlift/8boxer/original.ply")
# pcd

In [ ]:
# pcd_xyz = np.asarray(pcd.points)
# pcd_rgb = np.asarray(pcd.colors)
# full_pc = np.concatenate((pcd_xyz,pcd_rgb),axis=1)

In [ ]:
def distance(xi,xj):
  return np.exp(-1* np.linalg.norm(xi-xj))
# Adjacency Matrix
def pc_to_adj(pcd):
  A=[]
  for i in pcd.points:
    row=[]
    for j in pcd.points:
      row.append(distance(i,j))
    A.append(row)
  return A

In [ ]:
# original_pcd = o3d.io.read_point_cloud(path)
# A= distance_matrix(original_pcd.points[0:2048],original_pcd.points[0:2048])
# from scipy import sparse
# sA = sparse.csr_matrix(A)
# print(sA)

In [ ]:
# Degree Matrix
def adj_to_d(A):
  D=np.zeros([2048,2048])
  s=np.sum(np.asarray(A),axis=0)
  for i in range(0,2048):
    D[i][i]=np.sum(s[i])
  return D

In [ ]:
# Degree Matrix
# D= adj_to_d(A)

In [ ]:
# Graph laplacian 
#Lc = D - A

## Transforming the graph Signal from RGB to YUV :

In [ ]:
def rgb_to_yuv(rgb):
    y= 0.299 * rgb[0] + 0.587*rgb[1]+ 0.114 * rgb[2]
    u = 0.492 * (rgb[2]-y)
    v=0.877* (rgb[0]-y)
    return [y, u ,v]

In [ ]:
# yuv_colors=[]
# rgb_colors=pcd.colors
# for rgb in rgb_colors:
#     yuv_colors.append(rgb_to_yuv(rgb))

In [ ]:
# # Graph Signals : 
# Hy = np.array(yuv_colors)[:,0]
# Hu = np.array(yuv_colors)[:,1]
# Hv = np.array(yuv_colors)[:,2]

## Importing Data :  

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') 

In [ ]:
!rm -r /content/data/raht

In [ ]:
!mkdir /content/new_data

In [ ]:
!unrar x -Y "/content/gdrive/MyDrive/SKANDER/Desktop.rar" "/content/data/clean_train"

## Loss function :

In [ ]:
def custom_loss_function(y_true, y_pred):
  squared_difference = tf.square(y_true - y_pred)
  return tf.reduce_mean(squared_difference, axis=-1)

In [ ]:
def corrector(s):
    for a in range(0,len(s)):
        if (s[a]=='\''):
            s[a]='/'
    return s

## Graphs generation from point clouds : 

In [ ]:
!rm -r /content/spektral/datasets/PatchDatasets0
!rm -r /content/spektral/datasets/PatchDatasets1
!rm -r /content/spektral/datasets/PatchDatasets2
!mkdir spektral spektral/datasets
dataset=[]

mkdir: cannot create directory ‘spektral’: File exists
mkdir: cannot create directory ‘spektral/datasets’: File exists


In [ ]:
import time
t1 = time.perf_counter()
class PatchDatasets(Dataset):
    """
    A dataset for the graphs
    """
    def read(self):
      output = []
      # iter = 0
      for name in glob.glob('/content/spektral/datasets/PatchDatasets*/*.npz'):
        data = np.load(name,allow_pickle=True)
        output.append(
            Graph(x=data['x'], a=data['a'], y=data['y'])
        )
        # iter+=1
        # if(iter == 300):
        #   return output
      return output
    def download(self):
        # Create the directory
        if os.path.exists("/content/spektral/datasets/PatchDatasets/"):
          return 
        os.mkdir('/content/spektral/datasets/PatchDatasets0/')
        os.mkdir('/content/spektral/datasets/PatchDatasets1/')
        os.mkdir('/content/spektral/datasets/PatchDatasets2/')
        for name in glob.glob('/content/data/clean_train/predlift/*/r03.ply'):
            pcd = o3d.io.read_point_cloud(corrector(name))
            pcdy = o3d.io.read_point_cloud("/content/data/clean_train/predlift/"+name.split('/')[-2]+"/original.ply")
            rng=list(range(0,200,1))
            # Patch creation
            #while(i<=len(pcd.points)/2048):
            while(rng):
                nd = []
                rgb= []
                rgby=[]
                selected = random.randint(0, len(rng)-1)
                i=rng[selected]
                rng.pop(selected)
                if(i==int(len(pcd.points)/2048)):
                    nd = pcd.points[i*2048:]
                    rgb= pcd.colors[i*2048:]
                    rgby= pcdy.colors[i*2048:]
                    while(len(nd)!=2048):
                        nd.append(nd[-1])
                        rgb.append(rgb[-1])
                        rgby.append(rgby[-1])
                else:
                    nd = pcd.points[i*2048:(i+1)*2048]
                    rgb = pcd.colors[i*2048:(i+1)*2048]
                    rgby = pcdy.colors[i*2048:(i+1)*2048]
                
                # RGB to YUV : 
                yuv_colors=[]
                yuv_y_colors=[]
                for clr in range(0,len(rgb)):
                    yuv_colors.append(rgb_to_yuv(rgb[clr]))
                    yuv_y_colors.append(rgb_to_yuv(rgby[clr]))                     
                # Graph signals
                xY = np.array(yuv_colors)[:,0]
                xU = np.array(yuv_colors)[:,1]
                xV = np.array(yuv_colors)[:,2]

                yY = np.array(yuv_y_colors)[:,0]
                yU = np.array(yuv_y_colors)[:,1]
                yV = np.array(yuv_y_colors)[:,2]



                temp_pcd = o3d.geometry.PointCloud()
                temp_pcd.points = nd
                # Adjacency Matrix
                a =  distance_matrix(nd , nd)
                a =   np.exp(-1*a)
                # a= sparse.csr_matrix(A)
                # Saving patches to files
                path = "/content/spektral/datasets/PatchDatasets" + str(int(i/100))
                i+=1
                data_name= name.split('/')[-2]
                filenameY = os.path.join(path, f'graph_Y_{data_name}_{i}')
                filenameU = os.path.join(path, f'graph_U_{data_name}_{i}')
                filenameV = os.path.join(path, f'graph_V_{data_name}_{i}')
                np.savez(filenameU, x=xY, a=a, y=yY)


dataset = PatchDatasets()
t2 = time.perf_counter()
print('time taken to run:',t2-t1)

[Open3D WARNING] Read PLY failed: unable to read file: /content/data/clean_train/predlift/redblack/r03.ply
[Open3D WARNING] Read PLY failed: unable to read file: /content/data/clean_train/predlift/solider/r03.ply
[Open3D WARNING] Read PLY failed: unable to read file: /content/data/clean_train/predlift/longdress/r03.ply


/usr/local/lib/python3.7/dist-packages/spektral/data/graph.py:67: UserWarning: x was automatically reshaped to (2048, 1)
  warnings.warn(f"x was automatically reshaped to {x.shape}")


time taken to run: 369.3838868269995


In [ ]:
dataset

PatchDatasets(n_graphs=600)

## Model Creation/ Training : 

In [ ]:
#          Parameters 
dense_layers =[512 , 256 , 128 , 64 , 1]
cheb_layers = 64
qps = [51 , 46 , 40 , 34]
# Learning Rate
lr=1e-5
# Batch
batch_size = 8
# Adam
beta1 = 0.9
beta2=0.999
# Inputs
F = 1 # Features
N= 2048 # Patch
split = int(0.8 * len(dataset))

In [ ]:
def custom_loss_function(y_true, y_pred):
  squared_difference = tf.square(y_true - y_pred)
  return tf.reduce_mean(squared_difference, axis=-1)

class Predlift_Y_r1(Model):

    def __init__(self):
      super().__init__()
      self.graph_conv = ChebConv(channels=cheb_layers)
      self.graph_conv2 = ChebConv(channels=cheb_layers)
      self.gat_conv = GATConv(64)
      self.pool = GlobalSumPool()
      self.dense = Dense(dataset.n_labels, 'relu')

    def call(self, inputs):
      if len(inputs) == 2:
        x, a = inputs
      # Branch structure 
      out1 = self.graph_conv([x, a])
      out2 = self.gat_conv([out1, a])
      out3 = self.graph_conv2([out2, a])
      out4 = self.gat_conv([out3, a])
      out5 = self.graph_conv2([out4, a])
      out = tf.keras.layers.Concatenate(axis=1)([out1,out2,out3,out4,out5])
      # Dense Layers 
      out = self.pool(out)
      out = self.dense(out)
      # Residual learning
      x_res = x[:,:,0]
      out = tf.keras.layers.Add()([out, x_res])
      return out

t1 = time.perf_counter()

model = Predlift_Y_r1()
opt = keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=opt, loss=custom_loss_function)


# split = int(0.8 * len(dataset))

# data_tr, data_te = dataset[:split], dataset[split:]

loader_tr = BatchLoader(dataset, batch_size=8,  shuffle=True )

history_U_r1 = model.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=200)
t2 = time.perf_counter()
print('time taken to run:',t2-t1)

/usr/local/lib/python3.7/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'PatchDatasets' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


Epoch 1/200
75/75 [==============================] - 19s 212ms/step - loss: 383.4285
Epoch 2/200
75/75 [==============================] - 14s 193ms/step - loss: 237.3288
Epoch 3/200
75/75 [==============================] - 15s 194ms/step - loss: 155.8796
Epoch 4/200
75/75 [==============================] - 15s 195ms/step - loss: 106.4938
Epoch 5/200
75/75 [==============================] - 15s 195ms/step - loss: 74.7089
Epoch 6/200
75/75 [==============================] - 15s 195ms/step - loss: 53.4973
Epoch 7/200
75/75 [==============================] - 15s 196ms/step - loss: 38.9211
Epoch 8/200
75/75 [==============================] - 15s 195ms/step - loss: 28.6756
Epoch 9/200
75/75 [==============================] - 15s 195ms/step - loss: 21.3299
Epoch 10/200
75/75 [==============================] - 15s 195ms/step - loss: 15.9933
Epoch 11/200
75/75 [==============================] - 14s 193ms/step - loss: 12.0549
Epoch 12/200
75/75 [==============================] - 15s 194ms/step -

In [ ]:
import pandas as pd

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_U_r1.history) 

# save to json:  
hist_json_file = 'predlift_Y.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'predlift_Y.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
model.save("model_predlift_r03_Y_final")
# loaded_1 = keras.models.load_model(
#     'my_model_V', custom_objects={"Predlift_Y_r1": Predlift_Y_r1 ,"custom_loss_function":custom_loss_function}
# )
# import pickle
# from datetime import date 

# filename = 'model_Predlift_r01_Y_'+str(date.today())+'.sav'
# pickle.dump(model, open(filename, 'wb'))

/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [ ]:
loader_tr = BatchLoader(dataset, batch_size=8,  shuffle=True )

In [ ]:
loaded_1 = keras.models.load_model(
    '/content/model_Predlift_r03_Y_ckpt4(0-200)', custom_objects={"Predlift_Y_r1": Predlift_Y_r1(cheb_layers, dataset.n_labels) ,"custom_loss_function":custom_loss_function}
)

In [ ]:
history_y_r1 = loaded_1.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=200)

/usr/local/lib/python3.7/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'PatchDatasets' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


Epoch 1/200
75/75 [==============================] - 17s 175ms/step - loss: 6.5790e-04
Epoch 2/200
75/75 [==============================] - 12s 163ms/step - loss: 6.5788e-04
Epoch 3/200
75/75 [==============================] - 12s 164ms/step - loss: 6.5763e-04
Epoch 4/200
75/75 [==============================] - 12s 165ms/step - loss: 6.5786e-04
Epoch 5/200
75/75 [==============================] - 12s 166ms/step - loss: 6.5922e-04
Epoch 6/200
75/75 [==============================] - 12s 167ms/step - loss: 6.5759e-04
Epoch 7/200
75/75 [==============================] - 12s 163ms/step - loss: 6.5759e-04
Epoch 8/200
75/75 [==============================] - 12s 163ms/step - loss: 6.5759e-04
Epoch 9/200
75/75 [==============================] - 12s 164ms/step - loss: 6.5759e-04
Epoch 10/200
75/75 [==============================] - 12s 163ms/step - loss: 6.5759e-04
Epoch 11/200
75/75 [==============================] - 12s 162ms/step - loss: 6.5759e-04
Epoch 12/200
75/75 [=====================

In [ ]:
filename = '/content/model_Raht_r03_V_ckpt(0-100)'
loaded_1.save(filename)

In [ ]:
# !zip -r "/content/model_Predlift_r03_V_FINAL.zip" "/content/model_predlift_r03_V_final"
!zip -r "/content/model_Predlift_r03_U_FINAL.zip" "/content/model_predlift_r03_u_final"
# !zip -r "/content/model_Predlift_r03_Y_FINAL.zip" "/content/model_predlift_r03_Y_final"



  adding: content/model_predlift_r03_u_final/ (stored 0%)
  adding: content/model_predlift_r03_u_final/saved_model.pb (deflated 88%)
  adding: content/model_predlift_r03_u_final/keras_metadata.pb (deflated 81%)
  adding: content/model_predlift_r03_u_final/assets/ (stored 0%)
  adding: content/model_predlift_r03_u_final/variables/ (stored 0%)
  adding: content/model_predlift_r03_u_final/variables/variables.data-00000-of-00001 (deflated 22%)
  adding: content/model_predlift_r03_u_final/variables/variables.index (deflated 67%)
